<a href="https://colab.research.google.com/github/simulate111/General/blob/main/Turku_ERA5_Land_hourly_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install "cdsapi>=0.7.7"
!pip install netcdf4
!pip install netcdf4 h5netcdf

In [ ]:
import os

# Updated configuration: removed the UID prefix as per the error instructions
content = """url: https://cds.climate.copernicus.eu/api
key: c025f203-5930-4d9c-acd6-699c46be7fd8"""

with open(os.path.expanduser('~/.cdsapirc'), 'w') as f:
    f.write(content)

print("Configuration updated! Now attempting to update the library...")

# Also update your library to the latest version to match the new API
!pip install --upgrade cdsapi

In [ ]:
#@title 📍 Select City and View Map { run: "auto" }

# 1. Selection Menu
# Use the dropdown on the right to choose the city
selected_city = "Turku" #@param ["Turku", "Copenhagen", "Stockholm", "Oslo"]

# 2. City Coordinate Database
city_db = {
    "Turku": {"lat": 60.5, "lon": 22.3, "color": "red"},
    "Copenhagen": {"lat": 55.7, "lon": 12.6, "color": "blue"},
    "Stockholm": {"lat": 59.3, "lon": 18.1, "color": "green"},
    "Oslo": {"lat": 59.9, "lon": 10.7, "color": "purple"}
}

# Get current selection
coords = city_db[selected_city]
print(f"Current Selection: {selected_city} ({coords['lat']}, {coords['lon']})")

# 3. Generate Map showing all options
import folium
m = folium.Map(location=[59.0, 16.0], zoom_start=5, tiles='cartodbpositron')

for city, data in city_db.items():
    is_selected = (city == selected_city)
    folium.Marker(
        [data['lat'], data['lon']],
        popup=f"<b>{city}</b>",
        tooltip=city,
        icon=folium.Icon(color=data['color'], icon='star' if is_selected else 'info-sign')
    ).add_to(m)

m

In [1]:
import cdsapi
import os
import calendar

# Setting up the variables based on your selection above
LAT = city_db[selected_city]['lat']
LON = city_db[selected_city]['lon']
folder_city = selected_city.lower()

c = cdsapi.Client(progress=False)
tasks = [('2024', str(m).zfill(2)) for m in range(1, 13)] + [('2025', '01')]

print(f"--- 📥 Downloading data for: {selected_city} ---")

for i, (year, month) in enumerate(tasks):
    filename = f'{folder_city}_{year}_{month}.nc'

    if os.path.exists(filename):
        continue

    # Date logic for 2024 vs 2025
    last_day = 1 if year == '2025' else calendar.monthrange(int(year), int(month))[1]
    days = [str(d).zfill(2) for d in range(1, last_day + 1)]

    try:
        c.retrieve(
            'reanalysis-era5-land',
            {
                'variable': ['2m_temperature', '10m_u_component_of_wind',
                             '10m_v_component_of_wind', 'surface_solar_radiation_downwards'],
                'year': year, 'month': month, 'day': days,
                'time': [f"{str(h).zfill(2)}:00" for h in range(24)],
                'area': [LAT, LON, LAT, LON],
                'format': 'netcdf',
            },
            filename)
        print(f"✅ {filename} done.")
    except Exception as e:
        print(f"❌ Failed {year}-{month}: {e}")

print(f"--- 🎉 All {selected_city} downloads complete ---")

ModuleNotFoundError: No module named 'cdsapi'